# Web Page Data Extraction to Excel - Demo

This Python prototype automates the process of crawling a website, extracting product details, and saving them into an Excel file. It uses an LLM (via LangChain with Ollama- Gemma 2) to intelligently extract structured product data such as Product Name, MRP, and Selling Price from unstructured webpage text.

1. Start URL is defined and crawled recursively.

2. All page text is cleaned and chunked.

3. Each chunk is passed to an LLM prompt that extracts product data.

4. Valid JSONs are parsed and deduplicated.

5. Final output is saved to an Excel file.



In [1]:
%pip install requests bs4 langchain_ollama pandas

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'd:\python\copilot\myenv\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import re
import json
import pandas as pd
import numpy as np
from langchain_ollama import OllamaLLM


# ---------------------------------------
# 1. Web Crawler
# ---------------------------------------

In [3]:

visited = set()

def crawl(url, domain):
    """Recursively crawl a domain to collect text from pages."""
    if url in visited or domain not in url:
        return []
    visited.add(url)
    print(f"Crawling: {url}")

    try:
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.text, "html.parser")
        texts = [soup.get_text()]
        for link in soup.find_all("a", href=True):
            next_url = urljoin(url, link['href'])
            if domain in next_url:
                texts += crawl(next_url, domain)
        return texts
    except Exception as e:
        print(f"Error crawling {url}: {e}")
        return []



# ---------------------------------------
# 2. Text Cleaning
# ---------------------------------------

In [4]:

def clean_text(raw_text):
    """Clean raw HTML page text."""
    text = re.sub(r'<[^>]+>', '', raw_text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'not found 404 not found', '', text, flags=re.IGNORECASE)
    text = re.sub(r'loading\.\.\.|add to cart|shop now|my cart|login|register', '', text, flags=re.IGNORECASE)
    return text.strip()

def chunk_text(text, max_chars=2000):
    """Split large text into chunks for LLM input."""
    return [text[i:i + max_chars] for i in range(0, len(text), max_chars)]



# ---------------------------------------
# 3. LLM Processing
# ---------------------------------------

In [5]:

def extract_data_with_llm(chunks, model_name="gemma2:2b"):
    """Use Ollama LLM to extract structured data from text chunks."""
    llm = OllamaLLM(model=model_name, temperature=0)
    replies = []

    for chunk in chunks:
        prompt = f"""
Extract the following fields from the text: Product Name, MRP, and Selling Price.

Only respond with a **valid JSON object** in the exact format below, using actual values **from the text**:
{{
    "ProductName": "Exact product name from the text",
    "MRP": "₹[amount]",
    "sellingPrice": "₹[amount]"
}}

- If any value is missing or unclear, return null for that field.
- No explanations or extra text—just the JSON object.

Text:
\"\"\"{chunk}\"\"\"
"""
        reply = llm.invoke(prompt)
        replies.append(reply)
    return replies



# ---------------------------------------
# 4. JSON Cleaning and Parsing
# ---------------------------------------

In [6]:

def clean_json_string(raw_output):
    """Clean up extra formatting from LLM output."""
    return re.sub(r"```(json)?", "", raw_output).strip("` \n")

def parse_json_responses(replies):
    """Parse and filter valid JSON objects from LLM replies."""
    data = []
    for item in replies:
        cleaned = clean_json_string(item)
        try:
            parsed = json.loads(cleaned)
            if isinstance(parsed, dict):
                data.append(parsed)
        except json.JSONDecodeError as e:
            print("Error parsing JSON:", e)
            print("Problematic string:", cleaned)
    return data



# ---------------------------------------
# 5. Deduplication
# ---------------------------------------

In [7]:

def deduplicate_data(data):
    """Deduplicate product entries by name."""
    unique_data = {}
    for item in data:
        product_name = item.get("ProductName", "")
        if not isinstance(product_name, str) or not product_name.strip():
            continue
        product_name = product_name.strip()
        if product_name not in unique_data:
            unique_data[product_name] = item
    return list(unique_data.values())


# ---------------------------------------
# 6. Save to Excel
# ---------------------------------------

In [8]:
def save_to_excel(data, filename="products.xlsx"):
    """Save structured product data to Excel."""
    df = pd.DataFrame(data)
    df.replace(["null", "None", "", None], np.nan, inplace=True)
    df.dropna(subset=['ProductName'], inplace=True)
    df.to_excel(filename, index=False)
    return df


# ---------------------------------------
# Main Process
# ---------------------------------------

In [9]:

def main():
    start_url = "https://edenmart.in/"
    domain = urlparse(start_url).netloc

    # Step 1: Crawl website
    raw_texts = crawl(start_url, domain)

    # Step 2: Clean and chunk text
    full_cleaned_text = "\n---\n".join(clean_text(text) for text in raw_texts if text)
    chunks = chunk_text(full_cleaned_text)

    # Step 3: Extract structured data from chunks
    replies = extract_data_with_llm(chunks)

    # Step 4: Clean and parse JSON replies
    parsed_data = parse_json_responses(replies)

    # Step 5: Deduplicate by product name
    deduplicated = deduplicate_data(parsed_data)

    # Step 6: Save to Excel
    df = save_to_excel(deduplicated, filename="productsnew4.xlsx")

    print(df)

main()

Crawling: https://edenmart.in/
Crawling: https://edenmart.in
Crawling: https://edenmart.in/login
Crawling: https://edenmart.in/cart
Crawling: https://edenmart.in/allproductslist
Crawling: https://edenmart.in/about
Crawling: https://edenmart.in/contact
Crawling: https://edenmart.in/orders
Crawling: https://edenmart.in/index.html
Crawling: https://edenmart.in/addresslist
Crawling: https://edenmart.in/address
Crawling: https://edenmart.in/Shipping_Policy
Crawling: https://edenmart.in/Privacy_Policy
Crawling: https://edenmart.in/return_refund
Crawling: https://edenmart.in/terms_conditions
Crawling: https://edenmart.in/productdetail/1
Crawling: https://edenmart.in/productdetail/4
Crawling: https://edenmart.in/productdetail/5
Crawling: https://edenmart.in/productdetail/7
Crawling: https://edenmart.in/productdetail/8
Crawling: https://edenmart.in/productdetail/9
Crawling: https://edenmart.in/productdetail/10
Crawling: https://edenmart.in/productdetail/11
Crawling: https://edenmart.in/productd